In [1]:
import requests
import json
import pandas as pd
from opnieuw import retry
from requests.exceptions import HTTPError, ConnectionError
import time
from tinydb import TinyDB, Query

#### Functions

In [2]:
# Get all INE available operations
@retry(
    retry_on_exceptions=(ConnectionError, HTTPError),
    max_calls_total=4,
    retry_window_after_first_call_in_seconds=60,
)
def get_operaciones_disponibles():
    url = "https://servicios.ine.es/wstempus/js/ES/OPERACIONES_DISPONIBLES"
    payload = {}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code == 200:
        return response.json()


In [3]:
# Get data as series
@retry(
    retry_on_exceptions=(ConnectionError, HTTPError),
    max_calls_total=4,
    retry_window_after_first_call_in_seconds=60,
)
def get_datos(id_operacion, offset=1):
    url = f"https://servicios.ine.es/wstempus/js/ES/SERIES_OPERACION/{id_operacion}?page={offset}"
    payload = {}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code == 200:
        return response.json()

In [10]:
# Get data from a series
@retry(
    retry_on_exceptions=(ConnectionError, HTTPError),
    max_calls_total=4,
    retry_window_after_first_call_in_seconds=60,
)
def get_series(cod, nult=100):
    url = f"https://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{cod}?nult={nult}"
    payload = {}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    time.sleep(10)
    if response.status_code == 200:
        return response.json()


In [5]:
def iterate_operacion(id_operacion):
    responses = []
    offset = 1
    response = get_datos(id_operacion, offset)
    responses += response
    while True:
        offset += 1
        print(f'Iteracion en pagina {offset}')
        response = get_datos(id_operacion, offset)
        responses += response
        if len(response)<500:
            break
    return responses

#### Main

In [6]:
codigos=["CNE2444"]
for codigo in codigos: 
    db=TinyDB(r"C:\Users\anasa\Desktop\Ana\IRONHACK\Proyecto final/series.json")
    series_table=db.table("series")
    Serie=Query()
    for i in range(0, 5):
        try:
            print(f'Obteniendo datos de {codigo}. iteracion {i+1}')
            response=get_series(codigo)
            if response:
                if response.get('Data', []):
                    series_table.upsert(response, Serie.codigo==response["COD"])
                    break
        except:
            continue

Obteniendo datos de CNE2444. iteracion 1
Obteniendo datos de CNE2444. iteracion 2
Obteniendo datos de CNE2444. iteracion 3
Obteniendo datos de CNE2444. iteracion 4
Obteniendo datos de CNE2444. iteracion 5


In [11]:
get_series(codigo, 2)

{'COD': 'CNE2444',
 'Nombre': 'Total CNAE. Total Nacional. Valor añadido bruto. Precios corrientes. Variación anual. ',
 'FK_Unidad': 7,
 'FK_Escala': 13,
 'Data': []}